In [1]:
import pandas as pd

df = pd.read_csv('merged_df')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 632 entries, 0 to 631
Data columns (total 59 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   632 non-null    int64  
 1   experiment_id                632 non-null    object 
 2   management                   632 non-null    object 
 3   crop_type                    632 non-null    object 
 4   crop_season                  632 non-null    object 
 5   plant_family                 632 non-null    object 
 6   species                      632 non-null    object 
 7   species_mix                  632 non-null    object 
 8   sowing_date                  632 non-null    object 
 9   harvest_date                 632 non-null    object 
 10  biomass_seed                 632 non-null    float64
 11  plant_density                632 non-null    float64
 12  density_relative             632 non-null    float64
 13  density_factor      